In [1]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords

sep = '=========================================================================='
debug = False

# news_data = load_files("data/reviews/txt_sentoken")
news_data = load_files("data/news_articles/bbc")

# print(news_data.data[0])
# print(str(news_data.data[0])[1:])
# print(news_data.target)
# print(news_data.target_names)

unprocessed_text_data = news_data.data
labels = news_data.target
labelNames = news_data.target_names

print("init done")
print(sep)

print(np.sum(labels == 0))
print(np.sum(labels == 1))
print(np.sum(labels == 2))
print(np.sum(labels == 3))
print(np.sum(labels == 4))
print(labelNames)


if debug:
    unprocessed_text_data = [
        'Hi there',
        'Hello, there, you'
    ]



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bogdanisar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/bogdanisar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


init done
510
386
417
511
401
['business', 'entertainment', 'politics', 'sport', 'tech']


In [2]:
from sklearn.model_selection import train_test_split

trainData, testData, trainLabels, testLabels = train_test_split(unprocessed_text_data, labels, test_size=0.2, shuffle=True, random_state=0)

print("splitting done")
print(sep)

splitting done


In [3]:
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()

def cleanText(document):
    document = str(document)
    
    # Remove things like r'\n' r'\t'
    document = re.sub(r'\\\w', ' ', document)
    
    # Remove all the special characters
    document = re.sub(r'\W', ' ', document)
    
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    return document


clean_text_data = []
for idx in range(0, len(trainData)):
    clean_text_data.append( cleanText(trainData[idx]) )

# temp = r'hi\nth \t ere'
# print(temp)
# print(re.sub(r'\\\w', '', temp))

print(trainData[0])
print(sep)
print(clean_text_data[0])
print(sep)

# print(re.sub(r'\W', '', 'a ă . été / -- = + jj'))

print('cleaning text done')
print(sep)
unprocessed_text_data = None
    

b'Bid to cut court witness stress\n\nNew targets to reduce the stress to victims and witnesses giving evidence in courts in England and Wales have been announced by the lord chancellor.\n\nLord Falconer wants all crown courts and 90% of magistrates\' courts to have facilities to keep witnesses separate from defendants within four years. More video links will also be made available so that witnesses do not have to enter courtrooms. It is part of a five-year plan to help build confidence in the justice system.\n\nMinisters say the strategy is aimed at re-balancing the court system towards victims, and increasing the number of offenders brought to justice. Launching the Department for Constitutional Affairs\' plan, Lord Falconer said: "One of the top priorities will be a better deal for victims. "The needs and safety of victims will be at the heart of the way trials are managed.\n\n"Courts, judges, magistrates, prosecutors, police and victim support - all working together to ensure the ri

In [4]:
def getBagsOfWords(text_data):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.preprocessing import normalize

    if debug:
        min_df = 1
        max_df = 1.0
    else:
        max_df = 0.80
#         min_df = max(1, min(20, int(max_df * len(text_data)) - 1))
        min_df = 0.02
    
    vectorizer = CountVectorizer(strip_accents='ascii', min_df=min_df, max_df=max_df, max_features=1000)
    bow_data = vectorizer.fit_transform(text_data).toarray()
    
    return bow_data, vectorizer

bow_data, countVectorizer = getBagsOfWords(clean_text_data)

print( countVectorizer.stop_words_ )
print( len(countVectorizer.get_feature_names()) )
print( sorted(np.sum(bow_data, axis=0, dtype=np.int32)) )

countVectorizer.stop_words_ = None


print("bow done")
print(sep)
clean_text_data = None

{'gorillaz', '650', 'nicked', 'recording', 'millilitre', 'unpatriotic', 'crunch', 'ignorance', 'pied', 'aburizal', 'johannesson', 'ordered', 'specially', 'scrubbed', 'stylus', 'quicker', 'evil', 'fusilier', 'aara', '793', 'ejecting', 'a3507', 'mcfadden', 'rejoining', 'winfrey', 'impacted', 'firstsaver', 'sends', 'geneti', 'edward', 'uproar', 'passenger', 'nickname', 'scandalous', 'trinity', 'bolton', 'supposedly', 'bailout', 'investigated', 'adriatic', 'reignite', 'grandma', 'skilfully', 'yauch', 'dragoon', 'designer', 'caring', 'protegee', 'bechtler', 'owner', 'produced', 'caillet', 'galesi', 'mitigate', 'slows', 'eastenders', 'hansen', 'sachs', 'respite', 'broadus', 'az', 'hassell', '093', 'coupon', 'regulator', 'underrated', 'wef', 'coincided', 'bulletin', 'mine', 'floella', 'disengaged', 'delay', 'saracen', '050505', 'critically', 'enforceable', 'dinamo', 'frid', 'inspiration', 'amit', 'ricard', 'crushed', 'portrays', 'pakistani', 'finish', 'requisite', 'a386m', 'maciej', 'fossil',

In [5]:
def normalize(feature_data):
    from sklearn.preprocessing import normalize
    return normalize(feature_data, norm='l1')

# normalized_bow = normalize(bow_data)
normalized_bow = bow_data

if debug:
    print( np.array(list(bow_data)) )
    print( np.array(list(normalized_bow)) )
else:
    print(list(bow_data[0]))
    print(list(normalized_bow[0]))

    

print("normalization done")
print(sep)
bow_data = None

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 8, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
def get_tf_idf(feature_data):
    from sklearn.feature_extraction.text import TfidfTransformer
#     tfidfconverter = TfidfTransformer(smooth_idf=False, norm=None, use_idf=True)
    tfidfconverter = TfidfTransformer(smooth_idf=False, norm='l1', use_idf=True)
    tfidfdata = tfidfconverter.fit_transform(feature_data).toarray()
    
    return tfidfdata, tfidfconverter

tf_idf_data, tfidfconverter = get_tf_idf(normalized_bow)

if debug:
    print( np.array(list(normalized_bow)) )
    print( np.array(list(tf_idf_data)) )
else:
    print( list(tf_idf_data[0]) )



print("tf_idf-ization done")
print(sep)
normalized_bow = None

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0030486851611059283, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004004920195032272, 0.0, 0.0067508455269023335, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006259335600123986, 0.0, 0.0, 0.0, 0.007813778566961874, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005649206356258088, 0.0, 0.004917022747920094, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009506448635271772, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0041143803606814525, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006071635151651482, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004253090618384558, 0.016724979438913847, 0.0, 0.0, 0.0, 0.0, 0.002567311980037087, 0.0, 0.0, 0.0, 0.0, 0.005458644422157948, 0.0, 0.004602940548587419, 0.00514723714231641, 0.0, 0.006384858603281959, 0.0, 0.0, 0.006033124515377279, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
# classifier = SVC()
# classifier = LinearSVC()

classifier.fit(tf_idf_data, trainLabels)



print("classifier trained")
print(sep)

classifier trained


In [8]:
def getPredictionForData(data):
    for idx in range(0, len(data)):
        data[idx] = cleanText(data[idx])
    
#     print(data)
    
    data = countVectorizer.transform(data)
    
#     print(data.shape)
    
    data = normalize(data)
    
    print(data.shape)
    
    data = tfidfconverter.transform(data)
    return classifier.predict(data)

# predictedLabels = classifier.predict(testData)
predictedLabels = getPredictionForData(testData)
accuracy = np.average(testLabels == predictedLabels)
print(accuracy)



(445, 1000)
0.9550561797752809


In [9]:


politicalText = "The president was seen discussing the newest law passed in the parliament with his secretary of state. The senate agreed with the bill senator minister government"
entertainmentText = "The pop star just had her lastest album released. You can hear her sing the newest jingles at the concert this Easter's Eve. Santa Claus, video games, sitcom"
sportText = "Hockey, Basket, ball. Basketball, sport, ref, referee, goal, soccer, base, baseball, bat, cap, jersey, sweat, championship, league"

myTexts = [
    politicalText,
    entertainmentText,
    sportText,
    "bussiness, bussiness stock market lawyer CEO boss employee manager firm board owner founder bussiness",
    "algorithm, tech complexity ware advancements progress software",
    "gallery concert soccer musical book award audience film",
    "technology computer hardware software websites operate mail email program Microsoft Google PC performance",
    """
        He gave no further clues about its vintage, but the delicate, almost conversational vocals are reminiscent of his more recent live shows.

        Musically, it's spacious and haunting, with a dusting of piano, a sorrowful violin, and slow, brushed percussion - and while interpreting the lyrics would require weeks of scholarship, two lines in particular suggest Dylan may have chosen to release the song now as a commentary on US politics.

        "The day they killed him, someone said to me, 'Son, the age of the Antichrist has just only begun,'" he sings around the nine-minute mark.

        "I said, 'The soul of a nation's been torn away, and it's beginning to go into a slow decay.' And that it's 36 hours past judgement day." 
    """,
    
    """
        On Thursday top US share indexes capped their best three-day gains since the Great Depression.

        It comes as investors expect the US Congress to pass a massive stimulus package by the end of Saturday.

        The Group of 20 (G20) major economies has also pledged to inject over $5 trillion into the global economy.

        Japan's benchmark Nikkei 225 gained 1.6%, the Hang Seng in Hong Kong was up by 1.6% and China's Shanghai Composite rose 1%.

        That followed the Dow Jones Industrial Average and S&P 500 both climbing more than 6% on Wall Street, capping their best three-day streaks since the Great Depression of the 1930s. The tech-heavy Nasdaq ended higher for a second day, up 5.6%.
    """
]

myTextsPredictions = getPredictionForData(myTexts)
myTextsLabels = myTexts.copy()
for idx in range(len(myTextsLabels)):
    myTextsLabels[idx] = labelNames[myTextsPredictions[idx]]
print(myTextsPredictions, myTextsLabels, labelNames)
    

(9, 1000)
[2 3 3 0 3 1 4 1 0] ['politics', 'sport', 'sport', 'business', 'sport', 'entertainment', 'tech', 'entertainment', 'business'] ['business', 'entertainment', 'politics', 'sport', 'tech']


In [10]:
a1 = np.array([1,0,1,1])
a2 = np.array([0,0,1,1])
print(np.average(a1 == a2))
print(min(1,2,3))

0.75
1


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

str1 = "This is a test"
str2 = "Mary likes movies too. That is not a test"

vectorizer = CountVectorizer(strip_accents='ascii', min_df=0, max_df=1.0, max_features=None)
ws = vectorizer.fit_transform([str1,str2]).toarray()
# # print(vectorizer.vocabulary_)
# print(vectorizer.get_feature_names())
# print(vectorizer.get_stop_words())
# print(vectorizer.stop_words_)
print(ws)
# print(vectorizer.transform(['hello there, that is a cool test', "this is a"]).toarray())

# print(sep)


from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer(smooth_idf=False, norm='l1', use_idf=False)
tf_ws = tfidfconverter.fit_transform(ws).toarray()
print('norm=l1, use_idf=False', tf_ws, sep='\n')

tfidfconverter = TfidfTransformer(smooth_idf=False, norm=None, use_idf=True)
tf_ws = tfidfconverter.fit_transform(ws).toarray()
print('norm=None, use_idf=True', tf_ws, sep='\n')

tfidfconverter = TfidfTransformer(smooth_idf=False, norm='l1', use_idf=True)
tf_ws = tfidfconverter.fit_transform(ws).toarray()
print('norm=l1, use_idf=True', tf_ws, sep='\n')

tfidfconverter = TfidfTransformer(smooth_idf=False, norm='l1', use_idf=True)
tf_ws = tfidfconverter.fit_transform(normalize(ws)).toarray()
print('normalize and the tfidftransformer with norm=l1, use_idf=True', tf_ws, sep='\n')

# print(ws)

# ws = normalize(ws, norm='l1')
# print(ws)

# tfidfconverter = TfidfTransformer(smooth_idf=False, norm=None, use_idf=True)
# ws = tfidfconverter.fit_transform(ws).toarray()
# print(ws)


# print(sep)
# normalize_me = [1,1,0,1,1]
# print(normalize_me)
# print(normalize([normalize_me], norm='l1'))



[[1 0 0 0 0 1 0 1 0]
 [1 1 1 1 1 1 1 0 1]]
norm=l1, use_idf=False
[[0.33333333 0.         0.         0.         0.         0.33333333
  0.         0.33333333 0.        ]
 [0.125      0.125      0.125      0.125      0.125      0.125
  0.125      0.         0.125     ]]
norm=None, use_idf=True
[[1.         0.         0.         0.         0.         1.
  0.         1.69314718 0.        ]
 [1.         1.69314718 1.69314718 1.69314718 1.69314718 1.
  1.69314718 0.         1.69314718]]
norm=l1, use_idf=True
[[0.27077177 0.         0.         0.         0.         0.27077177
  0.         0.45845646 0.        ]
 [0.0822444  0.13925187 0.13925187 0.13925187 0.13925187 0.0822444
  0.13925187 0.         0.13925187]]
normalize and the tfidftransformer with norm=l1, use_idf=True
[[0.27077177 0.         0.         0.         0.         0.27077177
  0.         0.45845646 0.        ]
 [0.0822444  0.13925187 0.13925187 0.13925187 0.13925187 0.0822444
  0.13925187 0.         0.13925187]]
